# Aspect category classification using Deep Learning

In [1]:
import torch 
from torchtext.data import Field
from torchtext.data import Iterator , BucketIterator

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth' , -1)
import torch 
import torch.nn as nn

from torchtext import data
from torchtext import datasets

from torchtext.data import Field 
from torchtext.data import Iterator , BucketIterator
import torch.optim as optim
import os

In [3]:
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Changing column format to input it to TorchText data loader

In [4]:
tokenize = 'spacy'
TEXT = Field(sequential=True , tokenize=tokenize , lower=True , include_lengths=True)
LABEL = Field(sequential=False , use_vocab=False, dtype=torch.float) 

In [6]:
from torchtext.data import TabularDataset
 
# 'AMBIENCE', 'DRINKS', 'FOOD', 'LOCATION', 'RESTAURANT', 'SERVICE'
train_datafields = [ ('text' , TEXT) , ('AMBIENCE' , LABEL) , ('DRINKS' , LABEL) , 
                   ('FOOD' , LABEL) , ('LOCATION' , LABEL) , ('RESTAURANT', LABEL) , ('SERVICE' , LABEL) ]
train_data, valid_data  = TabularDataset.splits(
            path='.',  
            train='resturant_train_eng.csv',   
            validation='resturant_val_eng.csv',      
            format='csv',
            skip_header=True,  fields=train_datafields )


test_datafields =[ ('text' , TEXT) , ('AMBIENCE' , LABEL) , ('DRINKS' , LABEL) , 
                   ('FOOD' , LABEL) , ('LOCATION' , LABEL) , ('RESTAURANT', LABEL) , ('SERVICE' , LABEL) ]

test_data = TabularDataset(
            path = 'resturant_val_eng.csv' ,
            format='csv' , skip_header=True , 
            fields=test_datafields)

In [7]:
test_data[0].__dict__.keys()

dict_keys(['text', 'AMBIENCE', 'DRINKS', 'FOOD', 'LOCATION', 'RESTAURANT', 'SERVICE'])

In [9]:
np.array(test_data[1].__dict__)

array({'text': ['i', 'go', 'out', 'to', 'eat', 'and', 'like', 'my', 'courses', ',', 'servers', 'are', 'patient', 'and', 'never', 'rush', 'courses', 'or', 'force', 'another', 'drink', '.'], 'AMBIENCE': '0', 'DRINKS': '0', 'FOOD': '0', 'LOCATION': '0', 'RESTAURANT': '0', 'SERVICE': '1'},
      dtype=object)

#### We will use Glove word embeddings and keep max vocab size or around 4000

In [12]:
MAX_VOCAB_SIZE = 4000
BATCH_SIZE = 64
TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors =  "glove.42B.300d", 
                 unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)


 16%|█▌        | 299329/1917495 [00:26<02:20, 11480.21it/s]


 31%|███▏      | 601675/1917495 [00:52<01:59, 10995.69it/s]


 46%|████▋     | 889715/1917495 [01:18<01:29, 11542.65it/s]


 62%|██████▏   | 1184325/1917495 [01:44<01:03, 11498.84it/s]


 77%|███████▋  | 1479230/1917495 [02:10<00:37, 11563.14it/s]


 93%|█████████▎| 1775597/1917495 [02:36<00:12, 11659.07it/s]


100%|█████████▉| 1917436/1917495 [03:07<00:00, 11140.99it/s]

In [13]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of test examples: {len(test_data)}')

Number of training examples: 1366
Number of validation examples: 342
Number of test examples: 342


In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
train_iter, val_iter = BucketIterator.splits(
    (train_data, valid_data),
    batch_sizes=(BATCH_SIZE, BATCH_SIZE),
    device=device,
    sort_key=lambda x: len(x.text),
    sort_within_batch=True,
    repeat=False
)

test_iter = Iterator(test_data, batch_size=1, device=device, sort=False, sort_within_batch=False, repeat=False ,
                 train=False , shuffle=False )

In [19]:
class BatchWrapper:
    def __init__(self, iterator, x_var, y_vars):
        self.iterator, self.x_var, self.y_vars = iterator, x_var, y_vars
  
    def __iter__(self):
        for batch in self.iterator:
            x = getattr(batch, self.x_var)
            if self.y_vars is not None:
                y = torch.cat([getattr(batch, feat).unsqueeze(1) for feat in self.y_vars], dim=1).float()
            else:
                y = torch.zeros((1))
            yield (x, y)
    def __len__(self):
        return len(self.iterator)

train_loader = BatchWrapper(train_iter, "text", [ 'AMBIENCE', 'DRINKS', 'FOOD', 'LOCATION', 'RESTAURANT', 'SERVICE'])
valid_loader = BatchWrapper(val_iter, "text", ['AMBIENCE', 'DRINKS', 'FOOD', 'LOCATION', 'RESTAURANT', 'SERVICE'])
test_loader = BatchWrapper(test_iter, "text"   ,[ 'AMBIENCE', 'DRINKS', 'FOOD', 'LOCATION', 'RESTAURANT', 'SERVICE'])

In [20]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)        
        self.rnn = nn.LSTM(embedding_dim,  hidden_dim, num_layers=n_layers, 
                           bidirectional=bidirectional, dropout=dropout)
        self.fc = nn.Linear(hidden_dim*2 , output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence( embedded, text_lengths)
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        #output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)       
        hidden =(torch.cat((hidden[-2,:,:],  hidden[-1,:,:]), dim = 1))  
   
        return self.fc(hidden.squeeze(0))

In [30]:

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 100
OUTPUT_DIM = 6
N_LAYERS = 1
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [31]:
optimizer = optim.Adam(model.parameters() , lr = 1e-2)
loss_func = nn.BCEWithLogitsLoss()

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,148,706 trainable parameters


In [32]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.0072,  0.0380,  0.0675,  ...,  0.0396,  0.0018,  0.0760],
        [-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [-0.2084, -0.1493, -0.0175,  ..., -0.5407,  0.2120, -0.0094],
        ...,
        [ 0.1531, -0.0021, -0.0437,  ..., -0.0743,  0.3985, -0.0560],
        [-0.0619, -0.1963, -0.1243,  ..., -0.0344, -0.1365,  0.1695],
        [ 0.1952,  0.0509,  0.2714,  ..., -0.1678, -0.0798, -0.0308]])

In [33]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [34]:
model = model.to(device)
loss_func = loss_func.to(device)

In [35]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    acc = torch.sum(rounded_preds == y).float() 
    return acc

In [39]:
def train_model(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0    
    model.train()
    ct = 0
    for x, y in iterator:
        optimizer.zero_grad()
        text, text_lengths =x
        predictions = model(text, text_lengths)
        loss = criterion(predictions, y)
        acc = binary_accuracy(predictions, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        ct = ct + text.shape[1]   
    return epoch_loss / len(iterator), epoch_acc / (ct*6)

In [40]:
def validate_model(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0 
    ct = 0
    model.eval()
    with torch.no_grad():
        for x ,y  in iterator:
            text, text_lengths = x
            predictions = model(text, text_lengths)#.squeeze(1)
            loss = criterion(predictions,y)
            acc = binary_accuracy(predictions, y)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            ct = ct + text.shape[1]
        
    return epoch_loss / len(iterator), epoch_acc / (ct*6)

In [41]:
epochs = 5

best_valid_loss = float('inf')
best_epoch = 0
        
for epoch in range(1, epochs + 1):
    train_loss , train_acc = train_model(model, train_loader, optimizer, loss_func)
    valid_loss , valid_acc = validate_model(model, valid_loader, loss_func)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        best_epoch = epoch
        print(f'Best validation loss!! {best_valid_loss}')
        #torch.save(model.state_dict(), 'aspect_category.pt')
    #print(f'Epoch: {epoch}, Training Loss: {epoch_loss:.4f}, Validation Loss: {val_loss:.4f}')
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Best validation loss!! 0.2596495747566223
Epoch: 02
	Train Loss: 0.040 | Train Acc: 98.77%
	 Val. Loss: 0.260 |  Val. Acc: 92.30%
Epoch: 03
	Train Loss: 0.025 | Train Acc: 99.23%
	 Val. Loss: 0.268 |  Val. Acc: 92.20%
Epoch: 04
	Train Loss: 0.016 | Train Acc: 99.63%
	 Val. Loss: 0.296 |  Val. Acc: 92.06%
Epoch: 05
	Train Loss: 0.010 | Train Acc: 99.80%
	 Val. Loss: 0.323 |  Val. Acc: 91.62%
Epoch: 06
	Train Loss: 0.008 | Train Acc: 99.89%
	 Val. Loss: 0.314 |  Val. Acc: 91.96%


In [42]:
test_preds = []
text_data = []
true_label = []

epoch_loss = 0
epoch_acc = 0 
ct = 0
model.eval()
with torch.no_grad():
    for x ,y  in test_loader:
        text, text_lengths = x
        text_data.append(text.data.cpu().numpy())
        predictions = model(text, text_lengths)#.squeeze(1)
        rounded_preds = torch.round(torch.sigmoid(predictions))  #torch.round
        preds = rounded_preds.data.cpu().numpy()
        test_preds.append(preds)
        true_label.append(y.data.cpu().numpy())


In [43]:
aspects = ['AMBIENCE', 'DRINKS', 'FOOD', 'LOCATION', 'RESTAURANT', 'SERVICE']

In [49]:
"""
Merging prediction value with original test data and observe the metrics on overall level
"""
test_pred = pd.DataFrame(np.vstack(test_preds) ,index=valid_df.index , columns= [a+'_pred' for a in aspects])
test_aspect_pred = pd.merge(valid_df, test_pred , left_index=True ,right_index = True)

from sklearn.metrics import f1_score , confusion_matrix , accuracy_score , precision_score , recall_score , roc_auc_score

In [45]:
valid_df = pd.read_csv('resturant_val_eng.csv')

In [47]:
valid_df.tail(2)

,text,AMBIENCE,DRINKS,FOOD,LOCATION,RESTAURANT,SERVICE
340,The menu is very limited - i think we counted 4 or 5 entrees.,0,0,1,0,0,0
341,But that is highly forgivable.,0,0,0,0,0,1


In [52]:
test_aspect_pred.columns[-6:]

Index(['AMBIENCE_pred', 'DRINKS_pred', 'FOOD_pred', 'LOCATION_pred',
       'RESTAURANT_pred', 'SERVICE_pred'],
      dtype='object')

In [53]:
print("F1 score",f1_score(test_aspect_pred[aspects].as_matrix() , test_aspect_pred[test_aspect_pred.columns[-6:]].as_matrix() , average='macro' ))
print("Accuracy score" , np.mean(test_aspect_pred[aspects].as_matrix() == test_aspect_pred[test_aspect_pred.columns[-6:]].as_matrix()))

F1 score 0.6356208565371259
Accuracy score 0.9195906432748538


/home/FRACTAL/swati.tiwari/anaconda3/envs/fastai38/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/home/FRACTAL/swati.tiwari/anaconda3/envs/fastai38/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/FRACTAL/swati.tiwari/anaconda3/envs/fastai38/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [54]:
print('Accuracy for different aspect category ')
for col in aspects:
    print(col , ': ', test_aspect_pred[test_aspect_pred[col]==test_aspect_pred[col+'_pred']].shape[0]/test_aspect_pred.shape[0])

Accuracy for different aspect category 
AMBIENCE :  0.9152046783625731
DRINKS :  0.9824561403508771
FOOD :  0.9122807017543859
LOCATION :  0.9853801169590644
RESTAURANT :  0.8362573099415205
SERVICE :  0.8859649122807017


### Look for the accuracy of conflicting statements (sentiment changing within a sentence

In [31]:
test2_conflict = test_aspect_pred[test_aspect_pred['ind']==1]
print("F1 score",f1_score(test2_conflict[aspects].as_matrix() , test2_conflict[test2_conflict.columns[-5:]].as_matrix() , average='macro' ))
print("Accuracy score" , np.mean(test2_conflict[aspects].as_matrix() == test2_conflict[test2_conflict.columns[-5:]].as_matrix()))

F1 score 0.7492912295251918
Accuracy score 0.8368932038834952


/home/FRACTAL/swati.tiwari/anaconda3/envs/fastai38/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/home/FRACTAL/swati.tiwari/anaconda3/envs/fastai38/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [32]:
print('Accuracy for different aspect category ')
for col in aspects:
    print(col , ': ', test2_conflict[test2_conflict[col]==test2_conflict[col+'_pred']].shape[0]/test2_conflict.shape[0])

Accuracy for different aspect category 
ambience :  0.8349514563106796
anecdotes/miscellaneous :  0.7378640776699029
food :  0.8737864077669902
service :  0.8349514563106796
price :  0.9029126213592233


We can observe that accuracy and F1 score has improved significantly from TfIdf vector based classifier. We will use prediction of this model to find aspect categories in statement and then feed that to sentiment classifier

In [33]:
def aspect_pred(x):
    res = []
    for col in aspects:
        if x[col+'_pred']==1:
            res.append(col)
    return res

test_aspect_pred['aspects_pred'] = test_aspect_pred.apply(lambda x: aspect_pred(x) ,axis =1)
test_aspect_pred.drop(columns=['anecdotes/miscellaneous_pred', 'food_pred', 'service_pred', 'price_pred', 'ambience_pred'] , inplace=True)

In [34]:
def aspect_pred(x):
    res = []
    for col in aspects:
        if x[col]==1:
            res.append(col)
    return res

test_aspect_pred['aspects'] = test_aspect_pred.apply(lambda x: aspect_pred(x) ,axis =1)
test_aspect_pred.drop(columns=aspects , inplace=True)
test_aspect_pred.to_csv('../output/BiLSTM_aspect_prediction_test_data.csv' , index = False)

#### Using aspect prediction on test data , we have segmented some sentence using constituency parsing. Please go through: 2- Sentence segmentation using constituency parsing.ipynb

In [36]:
import ast
test_segments = pd.read_csv('../output/test_data_sentence_segmentation.csv')
for col in ['aspects_pred' , 'aspects' , 'splits']:
    test_segments[col]  = test_segments[col].apply(lambda x: ast.literal_eval(x))
    
test_segments['length_splits'] = test_segments['splits'].apply(lambda x: len(x))
test_segments.length_splits.value_counts()

1    598
2    45 
3    5  
4    1  
Name: length_splits, dtype: int64

In [38]:
single_aspect_df = test_segments[(test_segments['length_splits']==1)] # & (test_aspect_pred['ct_aspects']<=1)
multi_aspect_df = test_segments[test_segments['length_splits']>1]
multi_aspect_df.drop(columns=['aspects' , 'ct_aspects', 'length_splits' ] , inplace=True)
print('Reviews with only one aspect or no segment resulting from constituency parsing' , single_aspect_df.shape[0] )
print('Reviews with more than one sentence segment resulting from constituency parsing ' , multi_aspect_df.shape[0])

Reviews with only one aspect or no segment resulting from constituency parsing 598
Reviews with more than one sentence segment resulting from constituency parsing  51


For reviews with different segments mapping to multiple aspect categories, we will use pretrained Bi-LSTM model to predict most prominent aspect category in sentence split. 

In [39]:
multi_aspects_splits= []
for index, row in multi_aspect_df.iterrows():                   
    total_aspects = row['aspects_pred'].copy()
    total_sent = row['splits'].copy()    
    for sent in total_sent:
        multi_aspects_splits.append({  'id': index , 'sp_text': ' '.join(sent) , 'aspects_pred': total_aspects })

df_multi_aspects_splits = pd.DataFrame(multi_aspects_splits)
df_multi_aspects_splits[['sp_text','id']].to_csv('../output/test_data_split_ungrped.csv' , index=False)

In [40]:
test_datafields = [('sp_text' , TEXT) ]
test_data = TabularDataset( path = '../output/test_data_split_ungrped.csv' ,  format='csv' , skip_header=True ,  fields=test_datafields)
test_iter2 = Iterator(test_data, batch_size=1, device=device, sort=False, sort_within_batch=False, repeat=False , train=False , shuffle=False )
test_loader = BatchWrapper(test_iter2, "sp_text" , None )

In [41]:
test_preds = []
model.eval()
with torch.no_grad():
    for x ,y  in test_loader:
        text, text_lengths = x
        text_data.append(text.data.cpu().numpy())
        predictions = model(text, text_lengths)#.squeeze(1)
        rounded_preds = torch.round(torch.sigmoid(predictions))  #torch.round
        preds = rounded_preds.data.cpu().numpy()
        test_preds.append(preds)


In [42]:
testpreddf = pd.DataFrame(np.vstack(test_preds) ,index=df_multi_aspects_splits.index , columns= [a+'_pred' for a in aspects])
df_multi_aspects_splits = pd.merge(df_multi_aspects_splits, testpreddf , left_index=True ,right_index = True)

In [44]:
def aspect_pred(x):
    res = []
    for col in aspects: 
        if x[col+'_pred']==1:  res.append(col)
    return res

df_multi_aspects_splits['phrase_aspect'] = df_multi_aspects_splits.apply(lambda x: aspect_pred(x) ,axis =1)
df_multi_aspects_splits.drop(columns=['anecdotes/miscellaneous_pred', 'food_pred', 'service_pred', 'price_pred', 'ambience_pred'] , inplace=True)
df_multi_aspects_splits['remaining_aspect'] = df_multi_aspects_splits.apply(lambda x: list(set(x['aspects_pred']) - set(x['phrase_aspect']))  , axis =1)
df_multi_aspects_splits['remaining_aspect'] = df_multi_aspects_splits['remaining_aspect'].shift(1)

In [45]:
df_multi_aspects_splits['shift_id'] = df_multi_aspects_splits['id'].shift(1)
df_multi_aspects_splits.shift_id.fillna(0 , inplace = True)
df_multi_aspects_splits['shift_id'] =  df_multi_aspects_splits.apply(lambda x: 1 if x['id']!= x['shift_id']  else 0 , axis = 1 )
df_multi_aspects_splits['remaining_aspect']  = df_multi_aspects_splits.apply(lambda x: [] if x['shift_id']==1 else x['remaining_aspect'], axis = 1)

In [47]:
df_multi_aspects_splits2= []
for index, row in df_multi_aspects_splits.iterrows():                   
    total_aspects = row['phrase_aspect'].copy() 
    if len(total_aspects) ==1:
        df_multi_aspects_splits2.append({'id': row['id']  , 'sp_text':row['sp_text'] ,'fn_aspect': total_aspects[0] })
    else :
        if len(row['remaining_aspect']) ==1:
            df_multi_aspects_splits2.append({'id': row['id']  , 'sp_text':row['sp_text'] ,'fn_aspect': row['remaining_aspect'][0] })
        elif len(row['remaining_aspect']) ==0:
            for asp in row['phrase_aspect']:
                df_multi_aspects_splits2.append({'id': row['id'], 'sp_text':row['sp_text'],'fn_aspect':asp})
        else:
            for asp in row['remaining_aspect']:
                df_multi_aspects_splits2.append({'id': row['id'], 'sp_text':row['sp_text'],'fn_aspect':asp})
                

df1_aspect_lvl= pd.DataFrame(df_multi_aspects_splits2)
df1_aspect_lvl.shape

In [50]:
multi_aspect_df.head(2)

,text,ind,aspects_pred,splits
3,We were seated outside and the waiter spilled red wine and hot tea on myself and my date.,0,"[food, service]","[[We, were, seated, outside], [the, waiter, spilled, red, wine, and, hot, tea, on, myself, and, my, date]]"
4,"The crust is thin, the ingredients are fresh and the staff is friendly.",0,"[food, service]","[[The, crust, is, thin], [the, ingredients, are, fresh], [the, staff, is, friendly]]"


In [56]:
single_aspect= []
for index, row in single_aspect_df.iterrows():
    try:
        aspect = row['aspects_pred'][0]
    except:
        aspect =''
    single_aspect.append({'id': index, 'sp_text':row['text'],'fn_aspect':aspect})               

df2_aspect_lvl= pd.DataFrame(single_aspect)
df2_aspect_lvl.shape

(598, 3)

In [60]:
aspect_lvl_dataframe = pd.concat([df1_aspect_lvl , df2_aspect_lvl] , axis = 0)
aspect_lvl_dataframe.id.nunique()

In [66]:
aspect_lvl_dataframe.sort_values(by = ['id']  , inplace=True)
aspect_lvl_dataframe.reset_index(inplace= True , drop = True)

In [67]:
aspect_lvl_dataframe.to_csv('../output/aspect_level_test_data.csv' , index = False)